## Bulding Fast Queries on a csv file. 



## Dataset Description: An Inventory of Items available at a laptop store.

#### Columns Description:
1. ID: A unique identifier for the laptop.
2. Company: The name of the company that produces the laptop.
3. Product: The name of the laptop.
4. TypeName: The type of laptop.
5. Inches: The size of the screen in inches.
6. ScreenResolution: The resolution of the screen.
7. CPU: The laptop CPU.
8. RAM: The amount of RAM in the laptop.
9. Memory: The size of the hard drive.
10. GPU: The graphics card name.
11. OpSys: The name of the operating system.
12. Weight: The laptop weight.
13. Price: The price of the laptop

### Opening the file and previewing the data

In [16]:
import csv
with open("laptops.csv") as file1:
    read_content = csv.reader(file1)
    raw_data = list(read_content)
    headers = raw_data[0]
    data = raw_data[1:]
    print(headers, data[0:4])
    
##Note file automatically closed after 
##this code block completes

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price'] [['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']]


### Make a class which represents our inventory.

In [17]:
class Inventory():

    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            read_file =  csv.reader(file)
            data = list(read_file)
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])

### Create a class instance to validate that it's working as expected

In [19]:
laptop_inventory = Inventory('laptops.csv')
print(laptop_inventory.header)
print(len(laptop_inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


### Add a method to the inventory class to search laptops by ID

In [25]:
class Inventory():

    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            read_file =  csv.reader(file)
            data = list(read_file)
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None

### Validate the method is working 

In [27]:
laptop_instance = Inventory('laptops.csv')
print(laptop_instance.get_laptop_from_id('3362737'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]


### Optimize the lookup method by pre-processing the data.
The idea is preprocess the data into a dictionary where the keys are the IDs and the values the rows. Then, we will use that dictionary in the get_laptop_from_id() method. We can do this by:

1. Preprocess the data and create the dictionary in the __init__() method.
2. Re-implement the get_laptop_from_id() method. We will do it as a new method so that we can compare the two.

In [34]:
class Inventory():

    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            read_file =  csv.reader(file)
            data = list(read_file)
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
        

### Test the new method

In [35]:
laptop_instance = Inventory('laptops.csv')
print(laptop_instance.get_laptop_from_id_fast('3362737'))
print(laptop_instance.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


### Compare the performance of the 2 lookup methods:
- The get_laptop_from_id() method has time complexity O(R) where R is the number of rows.
- In contrast, the new implementation is time complexity O(1). It does so by using more memory to store the self.id_to_row dictionary and using a bit more time creating an instance (because it needs to create the dictionary).

#### Generate Random values to test with. (Using List Comprehenesion shorthand)

In [38]:
import time, random
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

#### Time to calculate values for slower lookup

In [39]:
laptop_instance = Inventory('laptops.csv')
total_time_no_dict = 0
for id in ids:
    start = time.time()
    laptop_instance.get_laptop_from_id(id)
    end = time.time()
    total_time_no_dict += end-start
print(total_time_no_dict)
    
    

1.7304761409759521


#### Time to calculate values for faster lookup

In [41]:
laptop_instance = Inventory('laptops.csv')
total_time_dict = 0
for id in ids:
    start = time.time()
    laptop_instance.get_laptop_from_id_fast(id)
    end = time.time()
    total_time_dict += end-start
print(total_time_dict)

0.005931377410888672


### Two Laptop Promotion: whenver a customer is issued a giftcard, the amount they receive should purchase exactly one or two laptops so that there is nothing leftover.
#### Add method to the class to implement this feature

In [46]:
class Inventory():

    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            read_file =  csv.reader(file)
            data = list(read_file)
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        ##First establish if exactly one laptop can be bought for dollars
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
        ##Second establish id exactly two laptops can be bought for dollars
        for i in self.rows:
            for j in self.rows:
                if i[-1] + j[-1] == dollars:
                    return True
        return False
        

#### Validate new method works as expected

In [47]:
laptop_instance = Inventory('laptops.csv')
print(laptop_instance.check_promotion_dollars(1000))
print(laptop_instance.check_promotion_dollars(442))


True
False


### Optimize the check_promotion_dollars method by preprocessing the data by storing all prices in a set when the class is initalized

In [49]:
class Inventory():

    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            read_file =  csv.reader(file)
            data = list(read_file)
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        #Check if one laptop equals dollars amount
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
        #Check if a two laptop combo equals dollars amount
        for i in self.rows:
            for j in self.rows:
                if i[-1] + j[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        #Check if one laptop equals dollars amount
        if dollars in self.prices:
            return True
        #Check if a two laptop combo equals dollars amount
        for price in self.prices:  
        #LOGIC: price2 = total(dollars) - price 1
            if dollars - price in self.prices:
                return True
        return False   

#### Validate new method works

In [51]:
laptop_instance = Inventory('laptops.csv')
print(laptop_instance.check_promotion_dollars_fast(1000))
print(laptop_instance.check_promotion_dollars_fast(442))

True
False


#### Compare Performance

In [56]:
prices = [random.randint(100, 5000) for _ in range(100)]
laptops_instance = Inventory('laptops.csv')

total_time_no_set = 0
for price in prices:
    start = time.time()
    laptops_instance.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end-start

total_time_set = 0
for price in prices:
    start = time.time()
    laptops_instance.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end-start

print(total_time_no_set, total_time_set)

2.7590436935424805 0.0010709762573242188


### Adding a method which utilizes a binary search to find all laptops within a given budget.
#### Given a budget of D dollars, find all laptops whose price it at most D

In [75]:
def row_price(row):
        return row[-1]
    
class Inventory():

    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            read_file =  csv.reader(file)
            data = list(read_file)
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key=row_price)
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        #Check if one laptop equals dollars amount
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
        #Check if a two laptop combo equals dollars amount
        for i in self.rows:
            for j in self.rows:
                if i[-1] + j[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        #Check if one laptop equals dollars amount
        if dollars in self.prices:
            return True
        #Check if a two laptop combo equals dollars amount
        for price in self.prices:  
        #LOGIC: price2 = total(dollars) - price 1
            if dollars - price in self.prices:
                return True
        return False  
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start
        

In [76]:
laptop_instance = Inventory('laptops.csv')
print(laptop_instance.find_first_laptop_more_expensive(1000))
print(laptop_instance.find_first_laptop_more_expensive(10000))

683
-1
